# General tasks group 23

## Task1

### Task1 - process data

> Create a distribution for the logistics delay of component "K7." Use the production date
(“Produktionsdatum”) from the dataset “Komponente_K7.csv” and the receiving date of
incoming goods (“Wareneingang”) from “Logistikverzug_K7.csv” (logistics delay). Assume
that produced goods are issued one day after the production date. For the model design in
Python, create a new dataset “Logistics delay” containing the required information from
both datasets

In [4]:
"""
Build a 'logistics_delay' table for component K7 (full outer join version).

- Reads the two input CSVs with automatic delimiter detection.
- Keeps all rows from both datasets, even if component_id does not match.
- Assumes goods are issued one day after the production date (if available).
- Computes logistics delay only where both production and receiving dates exist.
- Saves to case/data/logistics_delay.csv
"""

import os
import pandas as pd

SRC_COMPONENTS = "data/Logistikverzug/Komponente_K7.csv"
SRC_LOGISTICS = "data/Logistikverzug/Logistikverzug_K7.csv"
OUT_PATH = "additional_files/logistics_delay.csv"

os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)


def read_csv_smart(path: str) -> pd.DataFrame:
    """Read CSV with auto delimiter detection and basic cleanup."""
    df = pd.read_csv(path, sep=None, engine="python", dtype=str)

    # Drop unnamed index-like columns
    df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", case=False)]

    # Tidy headers
    df.columns = df.columns.str.strip().str.replace('"', '', regex=False)

    # Strip spaces/quotes only for string columns
    for col in df.columns:
        if pd.api.types.is_string_dtype(df[col]):
            df[col] = df[col].str.strip().str.strip('"')

    return df


def to_date(series: pd.Series) -> pd.Series:
    """Convert a Series to datetime, coercing errors to NaT."""
    return pd.to_datetime(series, errors="coerce")


def safe_series(df: pd.DataFrame, name: str, dtype: str = "object") -> pd.Series:
    """
    Return df[name] if present, else an NA/NaT series aligned to df.

    Args:
        df (pd.DataFrame): Input dataframe.
        name (str): Column name to fetch.
        dtype (str): Expected dtype ("object" or "datetime64[ns]").

    Returns:
        pd.Series: Either the existing column or a filled NA series.
    """
    if name in df.columns:
        return df[name]
    if dtype == "datetime64[ns]":
        return pd.Series(pd.NaT, index=df.index, dtype="datetime64[ns]")
    return pd.Series(pd.NA, index=df.index, dtype="object")


def coalesce_cols(df: pd.DataFrame, left: str, right: str,
                  dtype: str = "object") -> pd.Series:
    """
    Combine two columns into one, preferring left if not null.

    Args:
        df (pd.DataFrame): Input dataframe.
        left (str): Left column name.
        right (str): Right column name.
        dtype (str): Expected dtype.

    Returns:
        pd.Series: Coalesced series.
    """
    left_series = safe_series(df, left, dtype=dtype)
    right_series = safe_series(df, right, dtype=dtype)
    return left_series.where(left_series.notna(), right_series)


# ---- Load sources
components_df = read_csv_smart(SRC_COMPONENTS)
logistics_df = read_csv_smart(SRC_LOGISTICS)

# ---- Normalize headers
rename_map = {
    "IDNummer": "component_id",
    "Produktionsdatum": "production_date",
    "Wareneingang": "receiving_date",
    "Herstellernummer": "manufacturer_no",
    "Werksnummer": "plant_no",
    "Fehlerhaft": "defective_flag",
}
components_df = components_df.rename(
    columns={c: rename_map.get(c, c) for c in components_df.columns}
)
logistics_df = logistics_df.rename(
    columns={c: rename_map.get(c, c) for c in logistics_df.columns}
)

# ---- Parse dates
for df in (components_df, logistics_df):
    if "production_date" in df.columns:
        df["production_date"] = to_date(df["production_date"])
    if "receiving_date" in df.columns:
        df["receiving_date"] = to_date(df["receiving_date"])

# ---- Full outer join to keep all component IDs
merged_df = pd.merge(
    components_df.add_prefix("comp_"),
    logistics_df.add_prefix("logi_"),
    left_on="comp_component_id",
    right_on="logi_component_id",
    how="outer",
)

# ---- Coalesce core fields
component_id = coalesce_cols(
    merged_df, "comp_component_id", "logi_component_id", dtype="object"
)
manufacturer_no = coalesce_cols(
    merged_df, "comp_manufacturer_no", "logi_manufacturer_no", dtype="object"
)
plant_no = coalesce_cols(
    merged_df, "comp_plant_no", "logi_plant_no", dtype="object"
)
production_date = coalesce_cols(
    merged_df, "comp_production_date", "logi_production_date",
    dtype="datetime64[ns]"
)
receiving_date = coalesce_cols(
    merged_df, "comp_receiving_date", "logi_receiving_date",
    dtype="datetime64[ns]"
)

# Defective flag can appear in either file; keep whichever is not null
defective_flag_raw = coalesce_cols(
    merged_df, "comp_defective_flag", "logi_defective_flag", dtype="object"
)
defective_flag = pd.to_numeric(
    defective_flag_raw, errors="coerce"
).astype("Int64")

# ---- Compute derived fields
issue_date = production_date + pd.to_timedelta(1, unit="D")
logistics_delay_days = (receiving_date - issue_date).dt.days

# Extract last numeric token from component_id (e.g., K7-114-1142-3 -> 3)
component_number = (
    component_id.astype("string")
    .str.extract(r"(\d+)(?!.*\d)")[0]
    .astype("Int64")
)

# ---- Build final table
final_df = pd.DataFrame({
    "component_id": component_id,
    "component_number": component_number,
    "manufacturer_no": manufacturer_no,
    "plant_no": plant_no,
    "production_date": production_date,
    "issue_date": issue_date,
    "receiving_date": receiving_date,
    "logistics_delay_days": logistics_delay_days,
    "defective_flag": defective_flag,
})

# Keep all rows, sort for readability
final_df = final_df.sort_values(
    by=["component_id", "production_date", "receiving_date"]
).reset_index(drop=True)

# ---- Save
final_df.to_csv(OUT_PATH, index=False, encoding="utf-8")
print(f"Saved logistics delay table with {len(final_df)} rows to: {OUT_PATH}")

logistics_df = final_df


Saved logistics delay table with 306490 rows to: additional_files/logistics_delay.csv


### Task1 - a

> How is the logistics delay distributed? Justify your choice with statistical tests and
briefly describe your approach. (2 points)

In [16]:
import pandas as pd
import scipy.stats as stats


# Extract delay column
delays = logistics_df["logistics_delay_days"]

# 1. Descriptive statistics
mean_delay = delays.mean()
median_delay = delays.median()
std_delay = delays.std()
skewness = delays.skew()
kurtosis = delays.kurt()
max = delays.max()
min = delays.min()

print("Mean:", mean_delay)
print("Median:", median_delay)
print("Std Dev:", std_delay)
print("Skewness:", skewness)
print("Kurtosis:", kurtosis)
print("Max:", max)
print("Min:", min)

# 2. Normality test (Shapiro-Wilk)
shapiro_stat, shapiro_p = stats.shapiro(delays)
print("Shapiro-Wilk p-value:", shapiro_p)

# 3. Test against exponential distribution
# Estimate lambda = 1 / mean
lambda_est = 1 / mean_delay
ks_stat, ks_p = stats.kstest(delays, "expon", args=(0, 1/lambda_est))
print("KS test against Exponential, p-value:", ks_p)

Mean: 6.080436555841953
Median: 6.0
Std Dev: 1.0123016106982716
Skewness: 0.5674067497996634
Kurtosis: 0.6300548717524221
Max: 14
Min: 3
Shapiro-Wilk p-value: 0.0
KS test against Exponential, p-value: 0.0


/opt/anaconda3/envs/es/lib/python3.11/site-packages/scipy/stats/_morestats.py:1882: UserWarning:

p-value may not be accurate for N > 5000.



The logistics delay is not normally distributed. Descriptive statistics show a mean ≈ 6.08, median = 6.0, with slight positive skewness (0.57), indicating a right-skewed distribution. The Shapiro–Wilk test (p = 0.0) rejects normality, and the Kolmogorov–Smirnov test also rejects the exponential fit (p = 0.0). Therefore, the delays are best described as a slightly right-skewed discrete distribution, concentrated around 6 days, rather than normal or exponential.

### Task1 -b

> Determine the mean logistics delay, considering weekends. Interpret this number
and discuss possible alternatives. (2 points)

In [11]:
import pandas as pd
import numpy as np

# Convert dates to datetime format
logistics_df["issue_date"] = pd.to_datetime(logistics_df["issue_date"])
logistics_df["receiving_date"] = pd.to_datetime(logistics_df["receiving_date"])

# Function to calculate business days (excluding weekends)
def business_days_diff(start_date, end_date):
    # Use numpy busday_count to exclude weekends
    return np.busday_count(start_date.date(), end_date.date())

# Apply function to compute adjusted logistics delay
logistics_df["logistics_delay_business"] = logistics_df.apply(
    lambda row: business_days_diff(row["issue_date"], row["receiving_date"]),
    axis=1
)

# Calculate mean delay in business days (excluding weekends)
mean_business_delay = logistics_df["logistics_delay_business"].mean()

print("Mean business delay (excluding weekends):", mean_business_delay)


Mean business delay (excluding weekends): 4.343541387973507


The mean logistics delay is 4.34 business days, indicating that weekends account for much of the perceived delay. 

One may use calendar days to reflect customer waiting time, business days for operational efficiency, or a hybrid measure depending on the logistics schedule.

### Task1 -c 
Visualize the distribution appropriately by displaying the histogram and density
function using “plotly.” Describe how you selected the size of the bins. (2 points)

In [25]:
import numpy as np
import plotly.graph_objects as go
from scipy.stats import gaussian_kde

# Extract delay data
data = logistics_df["logistics_delay_days"].values

# Histogram trace (blue series)
hist = go.Histogram(
    x=data,
    xbins=dict(size=0.5),  # bin width
    name="Histogram (Count)",
    marker_color="rgba(0, 102, 204, 0.6)",  # medium blue, softer
    opacity=0.75,
    yaxis="y1"
)

# KDE calculation
kde = gaussian_kde(data)
x_range = np.linspace(2, 15, 200)  # smooth curve range
kde_y = kde(x_range)

# KDE trace (green series)
kde_line = go.Scatter(
    x=x_range,
    y=kde_y,
    mode="lines",
    name="KDE (Density)",
    line=dict(color="rgba(0, 153, 102, 1)", width=3),  # rich green
    yaxis="y2"
)

# Combine in one figure with dual y-axes
fig = go.Figure(data=[hist, kde_line])
fig.update_layout(
    title="Distribution of Logistics Delay Days",
    xaxis=dict(
        title="Delay Days",
        range=[2, 15],
        gridcolor="rgba(200,200,200,0.3)",  # light gray grid
        zerolinecolor="rgba(180,180,180,0.4)"  # softer zero line
    ),
    yaxis=dict(
        title=dict(text="Frequency (Count)", font=dict(color="rgba(0, 102, 204, 1)")),
        tickfont=dict(color="rgba(0, 102, 204, 1)"),
        gridcolor="rgba(200,200,200,0.3)",
        zerolinecolor="rgba(180,180,180,0.4)",
        side="left"
    ),
    yaxis2=dict(
        title=dict(text="Density", font=dict(color="rgba(0, 153, 102, 1)")),
        tickfont=dict(color="rgba(0, 153, 102, 1)"),
        overlaying="y",
        side="right"
    ),
    legend=dict(title="Legend"),
    template="plotly_white"
)

fig.show()


I selected the bin size of 0.5 based on the characteristics of the dataset. Since the mean of the logistics delays is around 6 with a relatively small standard deviation of about 1.02, the data are tightly clustered. A smaller bin size, such as 0.5, allows the histogram to capture meaningful variations without being overly fragmented, while still providing enough detail to show the distribution’s shape clearly.

### Task1 -d
> Describe the process for creating a decision tree to classify whether the
component (K7) is defective (Fehlerhaft) or not. (Hint: Use visualizations.) (2 points)

1. **Target & Features** – target: defective_flag; features: delay days, manufacturer, plant, etc.  

2. **EDA** – visualize class balance, delay distribution (histogram + KDE), and boxplots to explore patterns.  

3. **Preprocessing** – split into train/test sets, encode categorical features, and handle missing data.  

4. **Model (Decision Tree Details)**  - A decision tree can be built with machine learning using DecisionTreeClassifier, which splits data by features like delay days (e.g., if delay > 6 → defective; else → non-defective). 
```
   --> Decision Tree Model (eg)
          ├─ if delay > 6
          │      ├─ if manufacturer = 112 → Defective
          │      └─ else → Non-defective
          └─ else
                 ├─ if plant = 1132 → Non-defective
                 └─ else → Defective

```
  
5. **Visualization** – show the decision tree diagram (rules like “if delay > 7 then defective”), and a feature importance bar chart.  

6. **Evaluation** – use confusion matrix, ROC curve, accuracy, precision, recall, and F1-score to assess model performance.  

## Task 2

> Data Storage in Separate Files (2 Points) Explain why it makes sense to store the available data in separate files instead of
saving everything in one large table. Name at least four benefits. The available tables
represent a typical database structure. What is this structure called?

Storing the available data in separate files instead of saving everything in one large table has several benefits:

1. Reduces data redundancy by avoiding repeated storage of the same information.

2. Ensures data consistency because updates need to be made in only one place.

3. Improves query performance by allowing searches on smaller, focused tables.

4. Makes data maintenance and expansion easier by modifying or adding only the relevant tables.
   

This organization of data into multiple related tables is called a relational database structure.

## Task3

> Parts T16 in Registered Vehicles (3 Points) Determine how many parts T16 ended up in vehicles registered in Adelshofen.

In [ ]:
import os
import pandas as pd
from typing import Set, List

def safe_str(x) -> str:
    """Convert to string safely and return empty string for NaN-like values."""
    if isinstance(x, str):
        return x
    if pd.isna(x):
        return ""
    return str(x)


# Step 1: Load Zulassungen and filter vehicles registered in Adelshofen
zulassungen_path = "data/Zulassungen/Zulassungen_alle_Fahrzeuge.csv"
zulassungen_df = read_csv_smart(zulassungen_path)

# Normalize municipality names and collect vehicle IDs
zulassungen_df["Gemeinden_norm"] = zulassungen_df["Gemeinden"].str.strip().str.upper()
adelshofen_vehicles: Set[str] = set(
    safe_str(v) for v in zulassungen_df.loc[
        zulassungen_df["Gemeinden_norm"] == "ADELSHOFEN", "IDNummer"
    ].dropna().map(safe_str)
    if v
)

# Early exit if no vehicles
# print(f"Vehicles in Adelshofen: {len(adelshofen_vehicles)}")

# Step 2: From vehicle BOM tables, collect component IDs for those vehicles
fahrzeug_dir = "data/Fahrzeug"
fahrzeug_files = [
    os.path.join(fahrzeug_dir, f)
    for f in os.listdir(fahrzeug_dir)
    if f.lower().endswith(".csv")
]

# Only collect from component ID columns (start with "ID_" but exclude "ID_Fahrzeug")
component_ids: Set[str] = set()
for fpath in fahrzeug_files:
    df = read_csv_smart(fpath)
    if "ID_Fahrzeug" not in df.columns:
        continue

    # Filter rows for Adelshofen vehicles
    df_sub = df[df["ID_Fahrzeug"].map(safe_str).isin(adelshofen_vehicles)]
    if df_sub.empty:
        continue

    # Candidate component columns (ID_*) excluding vehicle id
    comp_cols: List[str] = [
        c for c in df_sub.columns
        if c.startswith("ID_") and c != "ID_Fahrzeug"
    ]

    # Some datasets also store component IDs in K*-prefixed columns
    k_cols: List[str] = [c for c in df_sub.columns if c.startswith("K") and "_" not in c]

    for col in comp_cols + k_cols:
        vals = df_sub[col].dropna().map(safe_str)
        for v in vals:
            if v:
                component_ids.add(v)

# Step 3: Walk through component detail tables, match to used components, count T16
komponente_dir = "data/Komponente"
komponente_files = [
    os.path.join(komponente_dir, f)
    for f in os.listdir(komponente_dir)
    if f.lower().endswith(".csv")
]

t16_count = 0

for fpath in komponente_files:
    df = read_csv_smart(fpath)

    # Skip files without T16 column
    if "ID_T16" not in df.columns:
        continue

    # Determine component prefix from filename, e.g., Bestandteile_Komponente_K1BE1.csv -> K1BE1
    fname = os.path.basename(fpath)
    try:
        comp_prefix = fname.rsplit("_", 1)[-1].replace(".csv", "").strip()
    except Exception:
        comp_prefix = ""

    if not comp_prefix:
        # Cannot infer component type; skip safely
        continue

    # The component ID column should be named like "ID_K1BE1"
    id_col = f"ID_{comp_prefix}"
    if id_col not in df.columns:
        # Fallback: try to guess by exact match among columns starting with "ID_"
        fallback_cols = [c for c in df.columns if c.startswith("ID_") and c != "ID_T16"]
        # Prefer a single column that ends with the prefix (exact component id column)
        exact = [c for c in fallback_cols if c.endswith(comp_prefix)]
        if exact:
            id_col = exact[0]
        else:
            # If still not found, we cannot match rows reliably; skip file
            continue

    # Relevant component IDs for this prefix (common format: 'K1BE1-...')
    relevant_ids = {cid for cid in component_ids if cid.startswith(f"{comp_prefix}-")}
    if not relevant_ids:
        continue

    # Filter rows where the component id is used in our vehicles
    df[id_col] = df[id_col].map(safe_str)
    used_rows = df[df[id_col].isin(relevant_ids)]

    if used_rows.empty:
        continue

    # Count rows that actually have T16 parts (non-null / non-empty)
    t16_in_used = used_rows["ID_T16"].map(safe_str).ne("").sum()
    t16_count += int(t16_in_used)

print("Number of T16 parts in Adelshofen vehicles:", t16_count)


Number of T16 parts in Adelshofen vehicles: 48


## Task 4 

> Attributes of the Registration Table (2 Points). Identify the data types of the attributes in the registration table
“Zulassungen_aller_Fahrzeuge.” Present your answers in a table integrated into your
Markdown document and describe the characteristics of the data types.

| Column        | Example value | Data type   | Explanation |
|---------------|--------------|-------------|-------------|
| *(unnamed)*   | 408097       | Number/Text | Just a row ID. Looks like a number, but could also be stored as text if leading zeros are important. |
| IDNummer      | 11-1-11-1    | Text        | A code with dashes. It’s not for math, so treat it as plain text. |
| Gemeinden     | DRESDEN      | Text (Category) | Town/municipality name. Only a limited set of values (e.g., cities). |
| Zulassung     | 2009-01-01   | Date        | A calendar date (year-month-day). No time info included. |


## Task 5

> Linear Model for Mileage. Create a linear model from the table “Fahrzeuge_OEM1_Typ11_Fehleranalyse”
relating mileage to suitable variables. Derive recommendations for OEM1 based on
this model.

Create Model:

In [29]:
# Step 1: Import required libraries
# We use pandas for data handling and scikit-learn for regression
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

# Step 2: Load the dataset
df = pd.read_csv("data/Fahrzeug/Fahrzeuge_OEM1_Typ11_Fehleranalyse.csv", sep=",")
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

# Step 3: Prepare features and target
# Dependent variable (y): mileage at failure ("Fehlerhaft_Fahrleistung")
# Independent variables (X): days, fuel, engine type
y = df["Fehlerhaft_Fahrleistung"]

# One-hot encode the categorical variable "engine"
encoder = OneHotEncoder(drop="first", sparse_output=False)
engine_encoded = encoder.fit_transform(df[["engine"]])

# Combine encoded engine with numerical features
X = pd.DataFrame(engine_encoded, columns=encoder.get_feature_names_out(["engine"]))
X = pd.concat([X, df[["days", "fuel"]]], axis=1)

# Step 4: Train linear regression model
model = LinearRegression()
model.fit(X, y)

# Step 5: Show coefficients
print("Intercept:", model.intercept_)
print("Coefficients:", dict(zip(X.columns, model.coef_)))


Intercept: 9590.351814446833
Coefficients: {'engine_medium': 7848.125372190192, 'engine_small': 11161.296725182996, 'days': -0.06057145448992347, 'fuel': 5235.171244927098}


Example: Predict mileage with trained model

In [30]:
sample = pd.DataFrame([{
    "engine": "large",
    "days": 1000,
    "fuel": 800
}])

# Encode engine and combine features
engine_encoded = encoder.transform(sample[["engine"]])
sample_X = pd.DataFrame(engine_encoded, columns=encoder.get_feature_names_out(["engine"]))
sample_X = pd.concat([sample_X, sample[["days", "fuel"]].reset_index(drop=True)], axis=1)

# Predict failure mileage
print("Predicted mileage:", model.predict(sample_X)[0])

Predicted mileage: 4197666.776301635


## Task 6 Hit and Run Accident Investigation (5 Points)

> Hit and Run Accident Investigation (5 Points). On 11.08.2010, there was a hit-and-run accident. The license plate of the car
involved is unknown. The police have asked for your assistance, as you work for the
Federal Motor Transport Authority, to find out where the vehicle with body part
number “K5-112-1122-79” was registered.

In [50]:
import pandas as pd

# Constants
TARGET_PART = "K5-112-1122-79"
ACCIDENT_DATE = pd.Timestamp("2010-08-11")

# Read and prepare Zulassungen
zulassung_df = read_csv_smart("data/Zulassungen/Zulassungen_alle_Fahrzeuge.csv")
zulassung_df.columns = [c.strip('"') for c in zulassung_df.columns]
zulassung_df["IDNummer"] = zulassung_df["IDNummer"].astype(str).str.strip('"')
zulassung_df["Gemeinden"] = zulassung_df["Gemeinden"].astype(str).str.strip('"')
zulassung_df["Zulassung"] = pd.to_datetime(zulassung_df["Zulassung"], errors="coerce")

# Load all parts files and combine
paths = [
    "data/Fahrzeug/Bestandteile_Fahrzeuge_OEM1_Typ11.csv",
    "data/Fahrzeug/Bestandteile_Fahrzeuge_OEM1_Typ12.csv",
    "data/Fahrzeug/Bestandteile_Fahrzeuge_OEM2_Typ21.csv",
    "data/Fahrzeug/Bestandteile_Fahrzeuge_OEM2_Typ22.csv",
]
parts_df = pd.concat([read_csv_smart(p) for p in paths], ignore_index=True)

# Find the vehicle ID by body part
hit = parts_df[parts_df["ID_Karosserie"] == TARGET_PART]
if not hit.empty:
    id_nummer = hit.iloc[0]["ID_Fahrzeug"].strip('"')
    valid = zulassung_df[
        (zulassung_df["IDNummer"] == id_nummer)
        & (zulassung_df["Zulassung"] <= ACCIDENT_DATE)
    ]
    if not valid.empty:
        row = valid.sort_values("Zulassung").iloc[0]
        print(
            f'Vehicle ID "{id_nummer}" registered in {row["Gemeinden"]}, \n'
            f'with body part "{TARGET_PART}" on {row["Zulassung"].date()}, \n'
            f'before the accident on 2010-08-11.'
        )
        
    else:
         print(f'Body part "{TARGET_PART}" was not found in the parts datasets.')
else:
    print(f'Body part "{TARGET_PART}" was not found in the parts datasets.')


Vehicle ID "12-1-12-82" registered in ASCHERSLEBEN, 
with body part "K5-112-1122-79" on 2009-01-02, 
before the accident on 2010-08-11.
